In [34]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import os
import json
import random
from sklearn.model_selection import train_test_split
from pandas_profiling import ProfileReport
from datetime import datetime
from collections import deque

## Paths and Data Files

In [18]:
os.getcwd()

'C:\\ALEX\\BGU\\TDA'

In [3]:
data_path = "sensors_data"
data_folder = os.path.join(os.getcwd(), data_path)
files1 = os.listdir(data_folder)
files2 = [os.path.join(data_folder, f) for f in files1 if f.endswith("_trim.tab")]
files = [f for f in files2 if os.path.isfile(f)]
delimiter = "\t"

In [4]:
files

['C:\\ALEX\\BGU\\TDA\\sensors_data\\fdc_F28_PROD_XEUS_trim.tab']

## Equipment digits explained:
### first digit is the XEUS NOD number:  1, 2, 3, or 4 otherwise
### second digit is the SITE within the NOD: 0, 1, 2.
### third digit is the entity number: 0 to 6 at most
### fourth digit is the side of the entity:  0 or 1

In [5]:
class Data_Preprocess:
    """
    Raw data is aggregated over 1hr time window, and each 1hr will be represented by mean/std-dev.
    """
    
    def __init__(self):
        self.df = None
        self.dfs = []
        
    def load_data(self, filename, delimiter):
        df = pd.read_csv(filename, delimiter=delimiter)
        print(f"File loaded with {len(df)} rows.")
        all_cols = True
        needed_cols = ['RUN_START_DATE', 'Equip', 'Feature', 'PREP_VALUE']
        cols = list(df.columns)
        miss_cols = []
        for col in needed_cols:
            if col not in cols:
                all_cols = False
                miss_cols.append(col)
        if len(miss_cols) > 0:
            print("Cound not find the following columns: " + ",".join(miss_cols))
        if all_cols:
            df['RUN_START_DATE'] = pd.to_datetime(df['RUN_START_DATE'])
            df = df.sort_values('RUN_START_DATE')
            df.insert(0, 'time', df['RUN_START_DATE'].dt.floor('H'))
            df['mean'] = df.groupby(['Equip', 'Feature'], as_index=False)['PREP_VALUE'].transform('mean')
            df['std'] = df.groupby(['Equip', 'Feature'], as_index=False)['PREP_VALUE'].transform('std')
            df['norm'] = (df['PREP_VALUE'] - df['mean']) / df['std']
            df = df.drop(['PREP_VALUE', 'mean', 'std'], axis=1)
            df = df.groupby(['time', 'RUN_START_WW', 'Equip', 'Feature'], as_index=False)['norm'].\
             agg(['mean', 'std']).reset_index().fillna(0)
            df = df.melt(id_vars=['time', 'RUN_START_WW', 'Equip', 'Feature'], value_vars=['mean', 'std'])
            df['series'] = df['Feature'] + "_" + df['variable']
            df = df.drop(['Feature', 'variable'], axis=1)
            self.df = df
    
    def prepare_series(self):
        self.df['key'] = self.df['Equip'] + "_" + self.df['series']
        uniq_keys = self.df['key'].unique()
        dfs = []
        for ukey in uniq_keys:
            print(f"Preparing key series {ukey} ...")
            ddf = self.df[self.df['key'] == ukey]
            ddf = ddf.sort_values(by=['series', 'Equip', 'time'], ascending=[True, True, True])
            ddf = ddf.reset_index(drop=True)
            ddf_list = []
            self.recur_split_series_no_multi_clusters(ukey, ddf, ddf_list, ave_size=20, threshold=1.8)
            for ddfl in ddf_list:
                ddfl = ddfl.sort_values(by=['series', 'Equip', 'time'], ascending=[True, True, True])
                ddfl = ddfl.reset_index(drop=True)
                m = np.mean(ddfl['value'])
                s = np.std(ddfl['value'])
                ddfl['norm'] = (ddfl['value'] - m) / s
                ddfl = ddfl.drop(['value'], axis=1)
                dfs.append(ddfl)
        self.dfs = dfs
    
    def recur_split_series_no_multi_clusters(self, ukey, df, ddf_list, ave_size=10, threshold=1.5):
        delta, indx = self.get_series_split_max_distance(df, ave_size=ave_size)
        if delta > threshold:
            print(f"Found two clusters or more and need split, in dataset {ukey}, indx {indx}, delta {delta}.")
            self.recur_split_series_no_multi_clusters(ukey, df[indx:], ddf_list, ave_size, threshold)
            self.recur_split_series_no_multi_clusters(ukey, df[:indx], ddf_list, ave_size, threshold)
        ddf_list.append(df)
        
    def get_series_split_max_distance(self, df, ave_size=10):
        delta = 0
        ki = 0
        for i in range(ave_size, len(df) - ave_size):
            k1 = np.mean(df[(i-ave_size+1):i]['value'].values)
            k2 = np.mean(df[i:(i+ave_size-1)]['value'].values)
            d = abs(k1 - k2)
            if d > delta:
                delta = d
                ki = i
        return delta, ki
    
    

In [6]:
data_obj = Data_Preprocess()
data_obj.load_data(files[0], delimiter)

File loaded with 11568827 rows.


In [7]:
len(data_obj.df)

892868

In [8]:
data_obj.prepare_series()

Preparing key series Equip2000_Feature2_mean ...
Preparing key series Equip2000_Feature5_mean ...
Preparing key series Equip2000_Feature6_mean ...
Preparing key series Equip2000_Feature7_mean ...
Preparing key series Equip2000_Feature8_mean ...
Preparing key series Equip2001_Feature2_mean ...
Preparing key series Equip2001_Feature5_mean ...
Preparing key series Equip2001_Feature6_mean ...
Preparing key series Equip2001_Feature7_mean ...
Preparing key series Equip2001_Feature8_mean ...
Preparing key series Equip2010_Feature0_mean ...
Found two clusters or more and need split, in dataset Equip2010_Feature0_mean, indx 2909, delta 3.273769076852412.
Found two clusters or more and need split, in dataset Equip2010_Feature0_mean, indx 2888, delta 2.4586883267019286.
Preparing key series Equip2010_Feature1_mean ...
Preparing key series Equip2010_Feature2_mean ...
Preparing key series Equip2010_Feature3_mean ...
Found two clusters or more and need split, in dataset Equip2010_Feature3_mean, indx

Preparing key series Equip2001_Feature9_mean ...
Preparing key series Equip2020_Feature0_mean ...
Found two clusters or more and need split, in dataset Equip2020_Feature0_mean, indx 377, delta 4.044147563764092.
Preparing key series Equip2020_Feature1_mean ...
Found two clusters or more and need split, in dataset Equip2020_Feature1_mean, indx 357, delta 6.437874497983519.
Found two clusters or more and need split, in dataset Equip2020_Feature1_mean, indx 20, delta 6.35043766542879.
Preparing key series Equip2020_Feature2_mean ...
Preparing key series Equip2020_Feature3_mean ...
Found two clusters or more and need split, in dataset Equip2020_Feature3_mean, indx 357, delta 6.063637492193319.
Found two clusters or more and need split, in dataset Equip2020_Feature3_mean, indx 37, delta 5.102636636922242.
Preparing key series Equip2020_Feature4_mean ...
Preparing key series Equip2020_Feature5_mean ...
Preparing key series Equip2020_Feature6_mean ...
Preparing key series Equip2020_Feature7_m

Preparing key series Equip2031_Feature4_std ...
Preparing key series Equip2031_Feature9_std ...
Preparing key series Equip2050_Feature0_std ...
Preparing key series Equip2050_Feature1_std ...
Preparing key series Equip2050_Feature2_std ...
Preparing key series Equip2050_Feature3_std ...
Preparing key series Equip2050_Feature4_std ...
Preparing key series Equip2050_Feature5_std ...
Preparing key series Equip2050_Feature6_std ...
Preparing key series Equip2050_Feature7_std ...
Preparing key series Equip2050_Feature8_std ...
Preparing key series Equip2050_Feature9_std ...
Preparing key series Equip2051_Feature0_std ...
Preparing key series Equip2051_Feature1_std ...
Preparing key series Equip2051_Feature2_std ...
Preparing key series Equip2051_Feature3_std ...
Preparing key series Equip2051_Feature4_std ...
Preparing key series Equip2051_Feature5_std ...
Preparing key series Equip2051_Feature6_std ...
Preparing key series Equip2051_Feature7_std ...
Preparing key series Equip2051_Feature8_

In [9]:
len(data_obj.dfs)

420

In [10]:
type(data_obj.dfs)

list

In [11]:
N = 0
for df in data_obj.dfs:
    N += len(df)
print(N)

1071834


In [12]:
np.mean(data_obj.dfs[5])

C:\ProgramData\Anaconda3_64bit\lib\site-packages\numpy\core\fromnumeric.py:3417: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
C:\ProgramData\Anaconda3_64bit\lib\site-packages\numpy\core\fromnumeric.py:3417: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


RUN_START_WW    2.022374e+05
norm            5.982616e-17
dtype: float64

In [13]:
np.std(data_obj.dfs[5])

C:\ProgramData\Anaconda3_64bit\lib\site-packages\numpy\core\fromnumeric.py:3558: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


time            43 days 14:42:36.826547264
RUN_START_WW                      6.238579
norm                                   1.0
dtype: object

In [17]:
data_obj.dfs[5].head(5)

,time,RUN_START_WW,Equip,series,key,norm
0,2022-06-26 23:00:00,202227,Equip2001,Feature2_mean,Equip2001_Feature2_mean,2.946667
1,2022-06-27 00:00:00,202227,Equip2001,Feature2_mean,Equip2001_Feature2_mean,1.225551
2,2022-06-27 01:00:00,202227,Equip2001,Feature2_mean,Equip2001_Feature2_mean,0.926174
3,2022-06-27 02:00:00,202227,Equip2001,Feature2_mean,Equip2001_Feature2_mean,-0.380895
4,2022-06-27 03:00:00,202227,Equip2001,Feature2_mean,Equip2001_Feature2_mean,-0.941805


In [15]:
for i in range(20):
    print(data_obj.dfs[i].loc[0, 'key'])

Equip2000_Feature2_mean
Equip2000_Feature5_mean
Equip2000_Feature6_mean
Equip2000_Feature7_mean
Equip2000_Feature8_mean
Equip2001_Feature2_mean
Equip2001_Feature5_mean
Equip2001_Feature6_mean
Equip2001_Feature7_mean
Equip2001_Feature8_mean
Equip2010_Feature0_mean
Equip2010_Feature0_mean
Equip2010_Feature0_mean
Equip2010_Feature0_mean
Equip2010_Feature0_mean
Equip2010_Feature1_mean
Equip2010_Feature2_mean
Equip2010_Feature3_mean
Equip2010_Feature3_mean
Equip2010_Feature3_mean


In [26]:
data_obj.dfs[5]['norm'].mean()

5.982616181790733e-17

In [28]:
data_obj.dfs[5]['norm']

0       2.946667
1       1.225551
2       0.926174
3      -0.380895
4      -0.941805
          ...   
3417   -0.586700
3418    0.526579
3419    0.542681
3420   -0.211703
3421    1.507836
Name: norm, Length: 3422, dtype: float64

In [33]:
data_obj.dfs[5]['norm'] = stats.zscore(data_obj.dfs[5]['norm'])

In [35]:
data_obj.dfs[5]['norm']

0       2.946667
1       1.225551
2       0.926174
3      -0.380895
4      -0.941805
          ...   
3417   -0.586700
3418    0.526579
3419    0.542681
3420   -0.211703
3421    1.507836
Name: norm, Length: 3422, dtype: float64

In [16]:
a = [1,2,3,4,5,6]
r = []
def recur_split(a, r):
    if len(a) > 1:
        recur_split(a[:1], r)
        recur_split(a[1:], r)
    if len(a) == 1:
        r.append(a)
print(a)
recur_split(a, r)
print(r)

[1, 2, 3, 4, 5, 6]
[[1], [2], [3], [4], [5], [6]]
